In [23]:
import pandas as pd

# 1. 데이터 불러오기
file_path = "./dataset/all_merged.csv"  # ← 파일 경로 확인해서 수정 가능
merged_df = pd.read_csv(file_path)

# 2. 선박별로 그룹화
grouped = merged_df.groupby("VSL_ID")

# 3. 좌표 저장용 리스트
first_coords = []
fifth_coords = []
tenth_coords = []

#  4. 각 선박별로 1, 5, 10번째 위치 추출
for vessel_id, group in grouped:
    if len(group) >=10:
        first_coords.append((group.iloc[0]['LAT'], group.iloc[0]['LON']))
        fifth_coords.append((group.iloc[4]['LAT'], group.iloc[4]['LON']))
        tenth_coords.append((group.iloc[9]['LAT'], group.iloc[9]['LON']))

# 5
def calc_mean(coords):
    lat_mean = sum([lat for lat, _ in coords]) / len(coords)
    lon_mean = sum([lon for _, lon in coords]) / len(coords)
    return lat_mean, lon_mean

# 6. 전체 평균 계산
first_avg = calc_mean(first_coords)
fifth_avg = calc_mean(fifth_coords)
tenth_avg = calc_mean(tenth_coords)

# 7. 결과 출력
print("🔹 1번째 위치 전체 평균:")
print("   위도:", round(first_avg[0], 4), "경도:", round(first_avg[1], 4))
print("🔹 5번째 위치 전체 평균:")
print("   위도:", round(fifth_avg[0], 4), "경도:", round(fifth_avg[1], 4))
print("🔹 10번째 위치 전체 평균:")
print("   위도:", round(tenth_avg[0], 4), "경도:", round(tenth_avg[1], 4))

🔹 1번째 위치 전체 평균:
   위도: 35.0943 경도: 129.0728
🔹 5번째 위치 전체 평균:
   위도: 35.0383 경도: 129.1103
🔹 10번째 위치 전체 평균:
   위도: 34.9901 경도: 129.099


In [27]:
import folium

# 1. folium 지도 객체 생성 (기본 중심은 부산 부근으로 설정)
m = folium.Map(location=[35.0, 129.1], zoom_start=8)

# 2. 위치별 평균 좌표 마커 추가
folium.Marker(
    location=[first_avg[0], first_avg[1]],
    popup="1번째 평균 위치",
    icon=folium.Icon(color="blue", icon="info-sign")
).add_to(m)

folium.Marker(
    location=[fifth_avg[0], fifth_avg[1]],
    popup="5번째 평균 위치",
    icon=folium.Icon(color="green", icon="info-sign")
).add_to(m)

folium.Marker(
    location=[tenth_avg[0], tenth_avg[1]],
    popup="10번째 평균 위치",
    icon=folium.Icon(color="red", icon="info-sign")
).add_to(m)

display(m)

# 도착항 평균내기 (항구마다)

In [32]:
import pandas as pd
import folium

# 1. 데이터 불러오기
merged_df = pd.read_csv("./dataset/all_merged.csv")

In [34]:
# 2. PORT_NAME 기준 그룹화
grouped_by_port = merged_df.groupby("PORT_NAME")

In [38]:
# 3. 좌표 저장용 리스트
port_last_coords = []
port_last_5_coords = []
port_last_10_coords = []

# 4. 각 도착항에서 마지막, -5, -10번째 위치 추출
for port_name, group in grouped_by_port:
    group_sorted = group.sort_values(by="TIMESTAMP")  # 시간 순 정렬
    if len(group_sorted) >= 10:
        last = group_sorted.iloc[-1]
        last_5 = group_sorted.iloc[-3]
        last_10 = group_sorted.iloc[-5]

        port_last_coords.append((port_name, last['LAT'], last['LON']))
        port_last_5_coords.append((port_name, last_5['LAT'], last_5['LON']))
        port_last_10_coords.append((port_name, last_10['LAT'], last_10['LON']))

# 5. 데이터프레임 변환
df_last = pd.DataFrame(port_last_coords, columns=["PORT_NAME", "LAT", "LON"])
df_last_5 = pd.DataFrame(port_last_5_coords, columns=["PORT_NAME", "LAT", "LON"])
df_last_10 = pd.DataFrame(port_last_10_coords, columns=["PORT_NAME", "LAT", "LON"])

# 6. 평균 계산
mean_last = df_last.groupby("PORT_NAME")[["LAT", "LON"]].mean().reset_index()
mean_last_5 = df_last_5.groupby("PORT_NAME")[["LAT", "LON"]].mean().reset_index()
mean_last_10 = df_last_10.groupby("PORT_NAME")[["LAT", "LON"]].mean().reset_index()

# 7. 지도 생성
m = folium.Map(location=[35.0, 129.0], zoom_start=5)

# 8. 마커 추가 함수
def add_markers(df, color, label):
    for _, row in df.iterrows():
        folium.Marker(
            location=[row["LAT"], row["LON"]],
            popup=f"{row['PORT_NAME']} ({label})",
            icon=folium.Icon(color=color)
        ).add_to(m)

# 9. 마커 추가 (색상별)
add_markers(mean_last, "red", "도착지점")         #  마지막 지점
add_markers(mean_last_5, "green", "-3 지점")     # 도착 3칸 전
add_markers(mean_last_10, "blue", "-5 지점")     # 도착 5칸 전
 
display(m)

In [5]:
import os
import pandas as pd
import numpy as np
from glob import glob

# 1. 경로 설정
mean_path = "./dataset/3rd_average.csv"
original_folder = "./dataset/original"
output_root = "./dataset/filtered_map"
os.makedirs(output_root, exist_ok=True)

# 2. 평균 좌표 불러오기
mean_df = pd.read_csv(mean_path)

# 3. 전체 선박 파일 순회
csv_files = glob(os.path.join(original_folder, "*", "*.csv"))

for file_path in csv_files:
    try:
        ship_df = pd.read_csv(file_path)

        if len(ship_df) < 2 or "LAT" not in ship_df.columns or "LON" not in ship_df.columns:
            continue

        # 4. 마지막 위경도로 도착항 추정
        last_lat, last_lon = ship_df.iloc[-3]["LAT"], ship_df.iloc[-3]["LON"]
        mean_df["distance"] = np.sqrt((mean_df["LAT"] - last_lat)**2 + (mean_df["LON"] - last_lon)**2)
        closest_port = mean_df.loc[mean_df["distance"].idxmin()]
        target_lat, target_lon = closest_port["LAT"], closest_port["LON"]
        port_name = closest_port["PORT_NAME"]

        # 5. 거리 계산 및 도착지점까지 유지
        ship_df["DIST_TO_PORT"] = np.sqrt((ship_df["LAT"] - target_lat)**2 + (ship_df["LON"] - target_lon)**2)
        closest_idx = ship_df["DIST_TO_PORT"].idxmin()
        filtered_df = ship_df.loc[:closest_idx]

        # 6. 저장 경로 생성 (항구별 폴더)
        port_folder = os.path.join(output_root, port_name)
        os.makedirs(port_folder, exist_ok=True)
        filename = os.path.basename(file_path)
        save_path = os.path.join(port_folder, filename)
        filtered_df.to_csv(save_path, index=False)

        print(f"저장 완료: {port_name}/{filename} ({len(ship_df) - len(filtered_df)}행 삭제됨)")

    except Exception as e:
        print(f"[오류 발생] {file_path} → {e}")


저장 완료: CNDAG/05e4d21a-31a0-3476-adf8-792558757ade.csv (2행 삭제됨)
저장 완료: CNDAG/42c578a0-ce4c-3cf7-977e-ece463542581.csv (6행 삭제됨)
저장 완료: CNDAG/a804036f-9377-3242-a11d-1b76dfa0dcec.csv (2행 삭제됨)
저장 완료: CNDAG/c4a0ea1d-e999-3b21-b394-521a19c74c8b.csv (3행 삭제됨)
저장 완료: CNHUA/028690b9-6aba-3401-8d39-d602f2a30d82.csv (2행 삭제됨)
저장 완료: CNHUA/091a653d-139a-33db-8f6d-2b5b389022d0.csv (3행 삭제됨)
저장 완료: CNHUA/0d13aad6-3c22-37e0-9106-76e8bb48e51e.csv (2행 삭제됨)
저장 완료: CNHUA/37154f3b-a9d2-36b4-bcf2-e8b9fcea9d11.csv (2행 삭제됨)
저장 완료: CNHUA/5b79af72-8160-3d81-b94d-a5dc3be324dd.csv (4행 삭제됨)
저장 완료: CNHUA/abefb79c-74ee-37f5-87fc-010b968f49fe.csv (5행 삭제됨)
저장 완료: CNHUA/c288770e-2988-3829-83df-3fb68d8dc1cd.csv (7행 삭제됨)
저장 완료: CNLYG/42c578a0-ce4c-3cf7-977e-ece463542581.csv (3행 삭제됨)
저장 완료: CNLYG/52a7cda9-1bcd-379d-a6dc-e1dbceb6c8b4.csv (0행 삭제됨)
저장 완료: CNLYG/615046a4-ea69-377a-b3a9-f9927165f9e4.csv (6행 삭제됨)
저장 완료: CNLYG/69d53028-a21c-3c8a-8a26-ec9ffc4bed0a.csv (3행 삭제됨)
저장 완료: CNLYG/771e4fc5-8f81-3f95-bc47-6fdc9d2c7dec.csv (

In [7]:
import os
import pandas as pd
import numpy as np
from glob import glob

# 1. 경로 설정
mean_path = "./dataset/3rd_average.csv"
original_folder = "./dataset/original"
output_root = "./dataset/filtered_map"
os.makedirs(output_root, exist_ok=True)

# 2. 평균 좌표 불러오기
mean_df = pd.read_csv(mean_path)
mean_df["LAT_5"] = mean_df["LAT"].round(5)
mean_df["LON_5"] = mean_df["LON"].round(5)

# 3. 전체 선박 파일 순회
csv_files = glob(os.path.join(original_folder, "*", "*.csv"))

MIN_KEEP_ROWS = 10

for file_path in csv_files:
    try:
        ship_df = pd.read_csv(file_path)

        if len(ship_df) < 2 or "LAT" not in ship_df.columns or "LON" not in ship_df.columns:
            continue

        # 4. 마지막 위경도 (소수점 5자리 반올림)
        last_lat = round(ship_df.iloc[-3]["LAT"], 5)
        last_lon = round(ship_df.iloc[-3]["LON"], 5)

        # 5. 도착항 추정 (소수점 5자리 기준으로 거리 계산)
        mean_df["distance"] = np.sqrt(
            (mean_df["LAT_5"] - last_lat) ** 2 + (mean_df["LON_5"] - last_lon) ** 2
        )
        closest_port = mean_df.loc[mean_df["distance"].idxmin()]
        target_lat, target_lon = closest_port["LAT"], closest_port["LON"]
        port_name = closest_port["PORT_NAME"]

        # 6. 거리 계산 및 유지 구간 선택
        ship_df["DIST_TO_PORT"] = np.sqrt((ship_df["LAT"] - target_lat) ** 2 + (ship_df["LON"] - target_lon) ** 2)
        closest_idx = ship_df["DIST_TO_PORT"].idxmin()

        if closest_idx < MIN_KEEP_ROWS:
            closest_idx = MIN_KEEP_ROWS - 1

        delete_ratio = (len(ship_df) - (closest_idx + 1)) / len(ship_df)
        if delete_ratio > 0.9:
            filtered_df = ship_df.copy()
        else:
            filtered_df = ship_df.loc[:closest_idx]

        # 7. 저장
        port_folder = os.path.join(output_root, port_name)
        os.makedirs(port_folder, exist_ok=True)
        filename = os.path.basename(file_path)
        save_path = os.path.join(port_folder, filename)
        filtered_df.to_csv(save_path, index=False)

        print(f"저장 완료: {port_name}/{filename} ({len(ship_df) - len(filtered_df)}행 삭제됨)")

    except Exception as e:
        print(f"[오류 발생] {file_path} → {e}")

저장 완료: CNDAG/05e4d21a-31a0-3476-adf8-792558757ade.csv (2행 삭제됨)
저장 완료: CNDAG/42c578a0-ce4c-3cf7-977e-ece463542581.csv (6행 삭제됨)
저장 완료: CNDAG/a804036f-9377-3242-a11d-1b76dfa0dcec.csv (2행 삭제됨)
저장 완료: CNDAG/c4a0ea1d-e999-3b21-b394-521a19c74c8b.csv (3행 삭제됨)
저장 완료: CNHUA/028690b9-6aba-3401-8d39-d602f2a30d82.csv (2행 삭제됨)
저장 완료: CNHUA/091a653d-139a-33db-8f6d-2b5b389022d0.csv (3행 삭제됨)
저장 완료: CNHUA/0d13aad6-3c22-37e0-9106-76e8bb48e51e.csv (2행 삭제됨)
저장 완료: CNHUA/37154f3b-a9d2-36b4-bcf2-e8b9fcea9d11.csv (2행 삭제됨)
저장 완료: CNHUA/5b79af72-8160-3d81-b94d-a5dc3be324dd.csv (4행 삭제됨)
저장 완료: CNHUA/abefb79c-74ee-37f5-87fc-010b968f49fe.csv (5행 삭제됨)
저장 완료: CNHUA/c288770e-2988-3829-83df-3fb68d8dc1cd.csv (7행 삭제됨)
저장 완료: CNLYG/42c578a0-ce4c-3cf7-977e-ece463542581.csv (3행 삭제됨)
저장 완료: CNLYG/52a7cda9-1bcd-379d-a6dc-e1dbceb6c8b4.csv (0행 삭제됨)
저장 완료: CNLYG/615046a4-ea69-377a-b3a9-f9927165f9e4.csv (6행 삭제됨)
저장 완료: CNLYG/69d53028-a21c-3c8a-8a26-ec9ffc4bed0a.csv (3행 삭제됨)
저장 완료: CNLYG/771e4fc5-8f81-3f95-bc47-6fdc9d2c7dec.csv (

# 도착항 제거 부분 확인 시각화

In [11]:
import folium
from folium.plugins import AntPath

def visualize_ship_track(ship_df, filtered_df, port_lat, port_lon, save_path="map.html"):
    # 지도 중심: 선박 경로 중간 지점
    center_lat = ship_df["LAT"].mean()
    center_lon = ship_df["LON"].mean()
    m = folium.Map(location=[center_lat, center_lon], zoom_start=8)

    # 전체 경로 (회색)
    folium.PolyLine(
        ship_df[["LAT", "LON"]].values,
        color="lightgray",
        weight=2,
        tooltip="전체 경로"
    ).add_to(m)

    # 유지된 구간 (파란색)
    folium.PolyLine(
        filtered_df[["LAT", "LON"]].values,
        color="blue",
        weight=3,
        tooltip="유지된 구간"
    ).add_to(m)

    # 삭제된 구간 (빨간색)
    if len(filtered_df) < len(ship_df):
        removed_df = ship_df.iloc[len(filtered_df):]
        folium.PolyLine(
            removed_df[["LAT", "LON"]].values,
            color="red",
            weight=3,
            tooltip="삭제된 구간"
        ).add_to(m)

    # 도착항 평균 위치 (초록색 마커)
    folium.Marker(
        [port_lat, port_lon],
        icon=folium.Icon(color="green", icon="flag"),
        popup="도착항 평균 좌표"
    ).add_to(m)

    # 저장
    display(m)